## Steps in this notebook:

#### Step 1: Load the model and the image

### Step 2: Create the Laplace mechanism function by modifying the original ML model's way of encoding the image

### Step 3: Plot different images (Original vs Encoded vs Encoded w. Noise) for CelebAHQ and PublicAHQ

In [1]:
import os
import sys
from PIL import Image,ImageShow
import numpy as np
import torch
import matplotlib.pyplot as plt
import yaml
from omegaconf import OmegaConf
import torchvision.transforms as transforms


from taming.models.cond_transformer import Net2NetTransformer

Get the dependencies

In [2]:
%pip install omegaconf>=2.0.0 pytorch-lightning>=1.0.8 einops transformers

sys.path.append(".")

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\DripTooHard\PycharmProjects\taming-transformers\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Prepare configurations of the CelebAHQ model as well as checkpoints.

In [3]:
#Prepare CelebAHQ configurations
config_path = r"C:\Users\DripTooHard\PycharmProjects\taming-transformers\scripts\2021-04-23T18-11-19-project.yaml"
celebAHQ_config = OmegaConf.load(config_path)
print(yaml.dump(OmegaConf.to_container(celebAHQ_config)))

#Init model with the chosen architecture and configurations
model = Net2NetTransformer(**celebAHQ_config.model.params)

data:
  params:
    batch_size: 24
    num_workers: 24
    train:
      params:
        size: 256
      target: taming.data.faceshq.CelebAHQTrain
    validation:
      params:
        size: 256
      target: taming.data.faceshq.CelebAHQValidation
  target: cutlit.DataModuleFromConfig
model:
  base_learning_rate: 0.0625
  params:
    cond_stage_config: __is_unconditional__
    first_stage_config:
      params:
        ddconfig:
          attn_resolutions:
          - 16
          ch: 128
          ch_mult:
          - 1
          - 1
          - 2
          - 2
          - 4
          double_z: false
          dropout: 0.0
          in_channels: 3
          num_res_blocks: 2
          out_ch: 3
          resolution: 256
          z_channels: 256
        embed_dim: 256
        lossconfig:
          target: taming.modules.losses.vqperceptual.DummyLoss
        n_embed: 1024
      target: taming.models.vqgan.VQModel
    first_stage_key: image
    transformer_config:
      params:
        bl

In [ ]:
#Load checkpoints
ckpt_path = r"C:\Users\DripTooHard\PycharmProjects\taming-transformers\scripts\taming-transformers\configs\last.ckpt"
sd = torch.load(ckpt_path, map_location="cpu")["state_dict"]
model.load_state_dict(sd)
missing, unexpected = model.load_state_dict(sd, strict=False)

In [ ]:
#Put model in evaluation mode
model.eval()
torch.set_grad_enabled(False)

## Load example data

Load an example segmentation and visualize.

In [ ]:
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image

# Define the function to load an image as a tensor
def load_image_tensor(path):
    image = Image.open(path)
    transform = transforms.Compose([
        transforms.PILToTensor()
    ])
    img_tensor = transform(image).permute(1,2,0).float()/255
    return img_tensor.permute(2, 0, 1).unsqueeze(0)

# Load the images
AHQ_example_path = r"C:\Users\DripTooHard\PycharmProjects\taming-transformers\scripts\taming-transformers\data\ffhq_images\01000\01000.jpg"
FFHQ_example_path = "C:/Users/DripTooHard/PycharmProjects/taming-transformers/scripts/taming-transformers/FFHQ_image.png"

AHQ_image = load_image_tensor(AHQ_example_path)
FFHQ_image = load_image_tensor(FFHQ_example_path)

# Plot the images with titles
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(AHQ_image[0].permute(1, 2, 0))
plt.title('CelebAHQ')
plt.axis('off')  # Hide the axis

plt.subplot(1, 2, 2)
plt.imshow(FFHQ_image[0].permute(1, 2, 0))
plt.title('FFHQ')
plt.axis('off')  # Hide the axis

plt.show()


# Step 2: We now add the possibility of encoding with the Laplace mechanism added before the quantization step.


In [ ]:
model.encode_to_z(AHQ_image,2)[0].shape

#Step 3:

We now recreate the images with noise and plot them next to eachother.


In [ ]:
epsilon = 0.1

def deconstruct_reconstruct(image, epsilon):
    image = image.type(torch.FloatTensor)  # Ensure the image tensor is of type FloatTensor
    image_zq, image_z_indices = model.encode_to_z(image, epsilon)
    image_recon = model.decode_to_img(image_z_indices, image_zq.shape)
    return image_recon

#Ignore this for now
def deconstruct_reconstruct_mix(image1,image2,epsilon):
    image1 = image1.type(torch.FloatTensor)  # Ensure the image tensor is of type FloatTensor
    image2 = image2.type(torch.FloatTensor)  # Ensure the image tensor is of type FloatTensor
    image_zq, image_z_indices = model.encode_to_z_mix(image1,image2, epsilon)
    image_recon = model.decode_to_img(image_z_indices, image_zq.shape)
    return image_recon


step_size = 0.01
max_range = 1

epsilon_values = np.arange(0,max_range,step_size)

folder = fr"C:\Users\DripTooHard\PycharmProjects\taming-transformers\scripts\Noisy Reconstructions\Laplace{max_range}{step_size}"
#os.mkdir(folder)


if(True):
    for epsilon in epsilon_values:
        # Assuming AHQ_image and FFHQ_image are defined and are tensors
        AHQ_recon = deconstruct_reconstruct(AHQ_image, epsilon)
        FFHQ_recon = deconstruct_reconstruct(FFHQ_image, epsilon)

        # Plot the images with titles
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(AHQ_recon[0].permute(1, 2, 0))
        plt.title('CelebAHQ-Reconstructed')
        plt.axis('off')  # Hide the axis

        # Use numpy to clip the values and ensure the correct data type for imsave

        plt.imsave(f"{folder}\AHQ\{epsilon}CelebAHQ-Reconstructed.png", AHQ_recon[0].permute(1, 2, 0).numpy().clip(0, 1))

        plt.subplot(1, 2, 2)
        plt.imshow(FFHQ_recon[0].permute(1, 2, 0))
        plt.title('FFHQ-reconstructed')
        plt.axis('off')  # Hide the axis

        # Use numpy to clip the values and ensure the correct data type for imsave
        plt.imsave(f"{folder}\FFHQ\{epsilon}FFHQ-reconstructed.png", FFHQ_recon[0].permute(1, 2, 0).numpy().clip(0, 1))


In [ ]:
np.dot([1,1,1,1],[1,1,1,1])
